In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
import pickle
import json
import ast
import astor
path = "/home/ubuntu/mydata/pkl_data/distributable/level4_inter"
dic = {}
with open(f"{path}/train.pkl", "rb") as f:
    dic["train"] = pickle.load(f)
with open(f"{path}/dev.pkl", "rb") as f:
    dic["dev"] = pickle.load(f)
with open(f"{path}/test.pkl", "rb") as f:
    dic["test"] = pickle.load(f)
src = pickle.load(open("/home/ubuntu/mydata/src_env.pkl", "rb"))
src = {x:y for x,y in src}

In [ ]:
cases = 0
def get_implementation(item, loc, no_comment=False, only_init=False):
    def check_ind(string):
        return len(string) - len(string.lstrip())
    file = loc["uri"][7:]
    try:
        with open(file, errors='ignore') as f:
            text = f.read()
    except:
        return ""
    
    lines = text.split("\n")
    be = loc["range"]["start"]['line']
    en = loc["range"]["end"]['line']
    if len(lines)<=be:
        print(loc["range"])
        print(file)
        print(item["name"])
        return ""
    
    ind = check_ind(lines[be])
    if not lines[be].strip().startswith("def ") and not lines[be].strip().startswith("class "):
        return ""
        
    lis = []
    for j in range(be, len(lines)):
        if len(lines[j].strip())==0: continue
        if check_ind(lines[j]) <= ind and j>be and not lines[j].strip().startswith(")"):
            break
        if only_init:
            if lines[be].strip().startswith("class "):
                if lines[j].strip().startswith("def ") and \
                    not lines[j].strip().startswith("def __init__") and check_ind(lines[j])==ind+4:
                    break
        if file == item["path"] and lines[be].find("def "+item["name"]+"(")!=-1 and j==item["position"]["line"]:
            global cases
            cases +=1 
            break
        lis.append(lines[j][ind:])

    if no_comment:
        for i in range(len(lis)):
            if i==0:
                body = "\n".join(lis)
            else:
                body = "\n".join(lis[:-i])
            try:
                node = ast.parse(body)
            except Exception as e:
                continue 
            node = node.body[0]
            if isinstance(node.body[0], ast.Expr):
                if hasattr(node.body[0], 'value') and isinstance(node.body[0].value, ast.Str):
                    node.body = node.body[1:]
            body = astor.to_source(node)
            return body
        
    return "\n".join(lis)
    
print(get_implementation(dic["train"][1863], dic["train"][1863]["definition"][0]))
print(get_implementation(dic["test"][0], dic["test"][0]["definition"][0]))
print(cases)

In [ ]:
from tqdm import tqdm
for k in dic:
    for x in tqdm(dic[k]):
        for loc in x["definition"]:
            new_body = get_implementation(x, loc)
            loc["body"] = new_body
print(cases)

In [ ]:
def check(x):
    flag = 0
    for loc in x["definition"]:
        if len(loc["body"].strip())==0: 
            continue
        #if loc["uri"].endswith("pyi"):
        #    continue
        flag +=1
    return flag

for k in dic:
    s = 0
    for i, x in enumerate(dic[k]):
        if check(x)==1:
            s+=1

    print(s)


In [ ]:
for k, lis in dic.items():
    new_lis = [x for x in lis if len(x["context"][0].strip())>0 and check(x)>0]
    print(len(new_lis), len(lis))
    with open(f"/home/ubuntu/mydata/pkl_data/distributable/level4_inter/{k}.pkl", "wb") as f:
        pickle.dump(new_lis, f)